# Data selection for hybrid perovskite single crystal data
    ## Author: Zhi Li
    ## Version 1.1
    ## 11-08-2018

In [1]:
import os, fnmatch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from mpl_toolkits.mplot3d import axes3d, Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
from matplotlib.colors import Normalize
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist, cdist, squareform
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.externals.six import StringIO
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import optunity
import optunity.metrics

## What amines we have tried?

In [2]:
file = "PostRunFulldataset.csv" # data file
df=pd.read_csv(file)
Inchi = pd.read_csv("Organic Inchikey.csv")
Inchidict = dict(zip(Inchi['Chemical Name'], Inchi['InChI Key (ID)']))

Amine_done = []
for i in df['_rxn_organic-inchikey']:
    if i not in Amine_done: 
        Amine_done.append(i)
Amine_number = len(Amine_done)
for i in range(Amine_number):
    Amine_done[i] = dict(zip(Inchi['InChI Key (ID)'],Inchi['Chemical Name']))[Amine_done[i]]
print (Amine_done)

['Phenethylammonium iodide', 'n-Butylammonium iodide', 'Ethylammonium Iodide', 'Methylammonium iodide', 'Guanidinium iodide', 'Acetamidinium iodide', 'Formamidinium Iodide', 'Imidazolium Iodide']


In [3]:
# function for specific amine, crystal class
def per3D(points, amine = None, category = None, \
          dim = ['_rxn_M_inorganic', '_rxn_M_organic', '_rxn_M_acid']):
    if amine is not None: amine = Inchidict[amine]
    if amine is None:
        if category is None:
            All_points = np.zeros((points.shape[0], 4))
            for i in range(3):
                All_points[:,i] = points[dim[i]]
            All_points[:,3] = points['_out_crystalscore']
        else:
            All_points = np.zeros((points[points['_out_crystalscore'] == category].shape[0], 4))
            for i in range(3):
                All_points[:,i] = points[dim[i]][points['_out_crystalscore'] == category]
            All_points[:,3] = points['_out_crystalscore'][points['_out_crystalscore'] == category]    
    else:
        if category is None:
            All_points = np.zeros((points[points["_rxn_organic-inchikey"] == amine].shape[0], 4))
            for i in range(3):
                All_points[:,i] = points[dim[i]][points["_rxn_organic-inchikey"] == amine]
            All_points[:,3] = points['_out_crystalscore'][points["_rxn_organic-inchikey"] == amine]
        else:
            All_points = np.zeros((points[points["_rxn_organic-inchikey"] == amine][points['_out_crystalscore'] == category].shape[0], 4))
            for i in range(3):
                All_points[:,i] = points[dim[i]][points["_rxn_organic-inchikey"] == amine][points['_out_crystalscore'] == category]
            All_points[:,3] = points['_out_crystalscore'][points["_rxn_organic-inchikey"] == amine][points['_out_crystalscore'] == category]
    
    return All_points

In [5]:
# Pick points with specific amine and class (category).
point_1 = per3D(df,amine = "Ethylammonium Iodide")

In [13]:
Ethyl_All = pd.DataFrame(point_1, columns = ["inorganic[M]", "organic[M]", "acid[M]", "class"])

In [14]:
Ethyl_All.to_csv("Ethyl_All.csv")